<a href="https://colab.research.google.com/github/sehan25/ESAA-2022-2/blob/main/Session/0905_%EB%B6%84%EB%A5%98_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 분류 연습 문제**
___
출처 : 핸즈온 머신러닝 Ch03 분류 연습문제 1, 2번

In [1]:
# import data
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1, as_frame = False)

In [2]:
X, y = mnist["data"], mnist["target"]

### **1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.**
___

1. `KNeghtborsClassifier`를 사용하는 것을 추천합니다. 
2. `weights`와 `n_neighbors` 하이퍼 파라미터로 그리드 탐색을 시도하여, 좋은 하이퍼 파라미터 값을 찾아보세요.

In [4]:
# import package
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [8]:
# Try GridSearch to optimize hyperparameter
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

params = [{'weights': ['uniform', 'distance'], 'n_neighbors': [3, 4, 5, 6]}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, params, cv=3, verbose=3)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3] END ....n_neighbors=3, weights=uniform;, score=0.969 total time=  59.0s
[CV 2/3] END ....n_neighbors=3, weights=uniform;, score=0.968 total time=  49.6s
[CV 3/3] END ....n_neighbors=3, weights=uniform;, score=0.968 total time=  48.7s
[CV 1/3] END ...n_neighbors=3, weights=distance;, score=0.970 total time=  48.4s
[CV 2/3] END ...n_neighbors=3, weights=distance;, score=0.969 total time=  48.2s
[CV 3/3] END ...n_neighbors=3, weights=distance;, score=0.969 total time=  48.0s
[CV 1/3] END ....n_neighbors=4, weights=uniform;, score=0.966 total time=  53.7s
[CV 2/3] END ....n_neighbors=4, weights=uniform;, score=0.966 total time= 1.1min
[CV 3/3] END ....n_neighbors=4, weights=uniform;, score=0.967 total time= 1.1min
[CV 1/3] END ...n_neighbors=4, weights=distance;, score=0.971 total time= 1.2min
[CV 2/3] END ...n_neighbors=4, weights=distance;, score=0.970 total time= 1.1min
[CV 3/3] END ...n_neighbors=4, weights=distance;,

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [3, 4, 5, 6],
                          'weights': ['uniform', 'distance']}],
             verbose=3)

In [9]:
# best hyperparameter
grid_search.best_params_

{'n_neighbors': 4, 'weights': 'distance'}

In [10]:
# best score
grid_search.best_score_

0.9703500000000002

In [11]:
# model test
from sklearn.metrics import accuracy_score

knn_clf2 = KNeighborsClassifier(n_neighbors=4, weights='distance')
knn_clf2.fit(X_train, y_train)
y_pred = knn_clf2.predict(X_test)
accuracy_score(y_test, y_pred)

0.9714

### **2. 다음 단계를 따라 인위적으로 훈련 세트를 늘리는 데이터 증식 또는 훈련 세트 확장 기법을 연습해봅시다.**
___

#### **STEP 1. MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어 보세요.**

In [12]:
from scipy.ndimage.interpolation import shift

In [13]:
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

####  **STEP 2. 앞에서 만든 함수를 이용하여, 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요**

In [14]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

In [16]:
import numpy as np

for dx, dy in ((1,0), (-1,0), (0,1), (0,-1)):
  for image, label in zip(X_train, y_train):
    X_train_augmented.append(shift_image(image, dx, dy))
    y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

####  **STEP 3. 위에서 확장한 데이터셋을 이용하여, 1번 문제에서 찾은 최적 모델을 훈련시키고, 테스트 세트에서 정확도를 측정해보세요**

In [17]:
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

In [22]:
knn_clf = KNeighborsClassifier(n_neighbors=4, weights='distance')
knn_clf.fit(X_train_augmented, y_train_augmented)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [20]:
y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9736